In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import time
from pyDOE import lhs
import matplotlib
# matplotlib.use('Agg')
# import imageio
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import shutil
import pickle
import scipy.io
import random
import math
from scipy.interpolate import griddata

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.optim import lr_scheduler

import warnings
warnings.filterwarnings("ignore")

In [2]:
import argparse
class Options_AC(object):
    def __init__(self):
        parser = argparse.ArgumentParser()
        parser.add_argument('--use_subnet', default=True, help=' use subnet or not')
        parser.add_argument('--no_cuda', action='store_true', default=False, help='disable CUDA or not')
        parser.add_argument('--dim_hidden', type=int, default=128, help='neurons in hidden layers')
        parser.add_argument('--hidden_layers', type=int, default=6, help='number of hidden layers')
        parser.add_argument('--lr', type=float, default=1e-3, help='initial learning rate')
        parser.add_argument('--epochs_Adam', type=int, default=100000, help='epochs for Adam optimizer')
        parser.add_argument('--epochs_LBFGS', type=int, default=0, help='epochs for LBFGS optimizer')
        parser.add_argument('--newton_iter', type=int, default=100, help='newton_iter for LBFGS optimizer')
        parser.add_argument('--step_size', type=int, default=5000, help='step size in lr_scheduler for Adam optimizer')
        parser.add_argument('--gamma', type=float, default=0.9, help='gamma in lr_scheduler for Adam optimizer')
        
        self.parser = parser

    def parse(self):
        arg = self.parser.parse_args(args=[])
        # arg.cuda = False
        # arg.device = torch.device('cpu')
        arg.cuda = not arg.no_cuda and torch.cuda.is_available()
        arg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        return arg

args = Options_AC().parse()
print(args.hidden_layers)

def save_model(state, is_best=None, save_dir=None):
    last_model = os.path.join(save_dir, 'last_model.pth')
    torch.save(state, last_model)
    if is_best:
        best_model = os.path.join(save_dir, 'best_model.pth')
        shutil.copyfile(last_model, best_model)

6


In [3]:
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    random.seed(seed)

def grad(outputs, inputs):
    """ compute the derivative of outputs associated with inputs

    Params
    ======
    outputs: (N, 1) tensor
    inputs: (N, D) tensor
    """
    return torch.autograd.grad(outputs, inputs,
                               grad_outputs=torch.ones_like(outputs),
                               create_graph=True)

def activation(name):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['leaky_relu', 'LeakyReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus()
    else:
        raise ValueError(f'unknown activation function: {name}')

In [4]:
seed_torch(42)

In [5]:
class Modified_MLP(nn.Module):
    def __init__(self, dim_in, dim_out, dim_hidden, hidden_layers,
                 act_name='tanh', init_name='xavier_normal'):
        super().__init__()
        self.hidden_layers = hidden_layers

        encoder_U = nn.Sequential()
        encoder_U.add_module('fc', nn.Linear(dim_in, dim_hidden, bias=True))
        encoder_U.add_module('act', activation(act_name))
        self.encoder_U = encoder_U

        encoder_V = nn.Sequential()
        encoder_V.add_module('fc', nn.Linear(dim_in, dim_hidden, bias=True))
        encoder_V.add_module('act', activation(act_name))
        self.encoder_V = encoder_V

        model = nn.Sequential()
        model.add_module('fc0', nn.Linear(dim_in, dim_hidden, bias=True))
        model.add_module('act0', activation(act_name))
        for i in range(1, hidden_layers):
            model.add_module(f'fc{i}', nn.Linear(dim_hidden, dim_hidden, bias=True))
            model.add_module(f'act{i}', activation(act_name))
        model.add_module(f'fc{hidden_layers}', nn.Linear(dim_hidden, dim_out, bias=True))

        self.model = model
        if init_name is not None:
            self.init_weight(init_name)

        self.size = self.model_size()

    def init_weight(self, name):
        """初始化网络参数"""

        if name == 'xavier_normal':
            nn_init = nn.init.xavier_normal_
        elif name == 'xavier_uniform':
            nn_init = nn.init.xavier_uniform_
        elif name == 'kaiming_normal':
            nn_init = nn.init.kaiming_normal_
        elif name == 'kaiming_uniform':
            nn_init = nn.init.kaiming_uniform_
        else:
            raise ValueError(f'unknown initialization function: {name}')

        for param in self.parameters():
            if len(param.shape) > 1:
                nn_init(param)
    
    def forward(self, x):
        """模型的正向传播"""
        U = self.encoder_U(x)
        V = self.encoder_V(x)
        for i in range(self.hidden_layers):
            x = self.model[2 * i](x)      # 调用线性层
            x = self.model[2 * i + 1](x)  # 调用激活层
            x = (1 - x) * U + x * V       # 特征融合
        x = self.model[-1](x)             # 调用最后一个线性层得到输出
        return x

    def model_size(self):
        """模型大小"""
        n_params = 0
        for param in self.parameters():
            n_params += param.numel()
        return n_params


## 网络模型(DNN)

In [6]:
class MLP(nn.Module):
    """Deep Neural Network"""

    def __init__(self, L_x, L_y, M_t, M_x, M_y, dim_hidden, hidden_layers, dim_out,
                 act_name='tanh', init_name='xavier_normal'):
        super().__init__()
        
#         dim_in = M_x * M_y * 4 + (M_x+M_y) * 2 + (M_t+1) + 1
        dim_in = (M_x+M_y) * 2 + (M_t+1) + 1
        
        model = DNN(dim_in, dim_out, dim_hidden, hidden_layers)
        self.model = model
        if init_name is not None:
            self.init_weight(init_name)
            
        self.M_t = M_t
        self.M_x = M_x
        self.M_y = M_y

        self.w_x = 2.0 * math.pi / L_x
        self.w_y = 2.0 * math.pi / L_y
        
        self.k_t = 10.0**torch.arange(0, M_t + 1)
        self.k_x = torch.arange(1, M_x+1).float()
        self.k_y = torch.arange(1, M_y+1).float()
        
        self.k_xx, self.k_yy = torch.meshgrid(self.k_x, self.k_y)
        self.k_xx = self.k_xx.reshape(-1)
        self.k_yy = self.k_yy.reshape(-1)
        
        self.k_t = nn.Parameter(self.k_t.float())
        self.k_x = nn.Parameter(self.k_x.float())
        self.k_y = nn.Parameter(self.k_y.float())
        self.k_xx = nn.Parameter(self.k_xx.float())
        self.k_yy = nn.Parameter(self.k_yy.float())
        
        
    def init_weight(self, name):
        if name == 'xavier_normal':
            nn_init = nn.init.xavier_normal_
        elif name == 'xavier_uniform':
            nn_init = nn.init.xavier_uniform_
        elif name == 'kaiming_normal':
            nn_init = nn.init.kaiming_normal_
        elif name == 'kaiming_uniform':
            nn_init = nn.init.kaiming_uniform_
        else:
            raise ValueError(f'unknown initialization function: {name}')

        for param in self.parameters():
            if len(param.shape) > 1:
                nn_init(param)
                
                           
    def input_encoding(self, t, x, y):
#         out = torch.hstack([torch.ones_like(t), self.k_t*t,
#                             torch.cos(self.k_x * self.w_x * x), torch.cos(self.k_y * self.w_y * y),
#                             torch.sin(self.k_x * self.w_x * x), torch.sin(self.k_y * self.w_y * y),
#                             torch.cos(self.k_xx * self.w_x * x) * torch.cos(self.k_yy * self.w_y * y),
#                             torch.cos(self.k_xx * self.w_x * x) * torch.sin(self.k_yy * self.w_y * y),
#                             torch.sin(self.k_xx * self.w_x * x) * torch.cos(self.k_yy * self.w_y * y),
#                             torch.sin(self.k_xx * self.w_x * x) * torch.sin(self.k_yy * self.w_y * y)])
        
        out = torch.hstack([torch.ones_like(t), self.k_t*t,
                            torch.cos(self.k_x * self.w_x * x), torch.cos(self.k_y * self.w_y * y),
                            torch.sin(self.k_x * self.w_x * x), torch.sin(self.k_y * self.w_y * y)])
        
        return out    
            
                           
    def forward(self, H):
        t = H[:, 0:1]
        x = H[:, 1:2]
        y = H[:, 2:3]
    
        # embedding
        H = self.input_encoding(t, x, y)
        
        # 特征融合
        H = self.model(H)
        return H
    
    def forward_test(self, x):
        print(f"{'input':<20}{str(x.shape):<40}")
        for name, module in self.model._modules.items():
            x = module(x)
            print(f"{name:<20}{str(x.shape):<40}")
        return x

    def model_size(self):
        n_params = 0
        
        for param in self.parameters():
            n_params += param.numel()
        return n_params

In [7]:
class DNN(nn.Module):
    """Deep Neural Network"""

    def __init__(self, dim_in, dim_out, dim_hidden, hidden_layers,
                 act_name='tanh', init_name=None):
        super().__init__()
        model = nn.Sequential()

        model.add_module('fc0', nn.Linear(dim_in, dim_hidden, bias=True))
        model.add_module('act0', activation(act_name))
        for i in range(1, hidden_layers):
            model.add_module(f'fc{i}', nn.Linear(dim_hidden, dim_hidden, bias=True))
            model.add_module(f'act{i}', activation(act_name))
        model.add_module(f'fc{hidden_layers}', nn.Linear(dim_hidden, dim_out, bias=True))

        self.model = model
        if init_name is not None:
            self.init_weight(init_name)

    def init_weight(self, name):
        if name == 'xavier_normal':
            nn_init = nn.init.xavier_normal_
        elif name == 'xavier_uniform':
            nn_init = nn.init.xavier_uniform_
        elif name == 'kaiming_normal':
            nn_init = nn.init.kaiming_normal_
        elif name == 'kaiming_uniform':
            nn_init = nn.init.kaiming_uniform_
        else:
            raise ValueError(f'unknown initialization function: {name}')

        for param in self.parameters():
            if len(param.shape) > 1:
                nn_init(param)

    def forward(self, x):
        return self.model(x)

    def forward_test(self, x):
        print(f"{'input':<20}{str(x.shape):<40}")
        for name, module in self.model._modules.items():
            x = module(x)
            print(f"{name:<20}{str(x.shape):<40}")
        return x

    def model_size(self):
        n_params = 0
        for param in self.parameters():
            n_params += param.numel()
        return n_params

In [8]:
# model = MLP(L_x=1, L_y=1, M_t=2, M_x=5, M_y=5, dim_hidden=128, hidden_layers=6, dim_out=1)
model = DNN(dim_in=4, dim_out=1, dim_hidden=128, hidden_layers=6)
model.cuda()
args.model=model

AssertionError: Torch not compiled with CUDA enabled

## 数据集生成

In [9]:
class Trainset_AC():
    def __init__(self, *args):
        self.args = args
        self.shape = (self.args[1], self.args[0])
        
    def __call__(self):
        return self.data()
    
    def data(self):
        
        Nsd = self.args[0]
        n_ics = self.args[1]
        
        lb = np.array([0.0, 0.0, 0.0, 0.0])
        ub = np.array([1., 1., 1., 1.])
        txy = (ub-lb)*lhs(4, Nsd)+lb
        
        lb_ics = np.array([0.0, 0.0, 0.0, 0.0])
        ub_ics = np.array([0.0, 1., 1., 1.]) 
        txy_ics = (ub_ics-lb_ics)*lhs(4, n_ics)+lb_ics
        
        u_ics = np.tanh((0.35-np.sqrt((txy_ics[:,[1]]-0.5)**2 + (txy_ics[:,[2]]-0.5)**2) + (txy_ics[:,[3]]-0.5)**2)/(2*0.05))
        
        txy = torch.from_numpy(txy).float().cuda()
        txy_ics = torch.from_numpy(txy_ics).float().cuda()
        u_ics = torch.from_numpy(u_ics).float().cuda()
        
        return txy, txy_ics, u_ics

In [10]:
trainset = Trainset_AC(25600, 1024)
args.trainset = trainset
txy, txy_ics, u_ics = trainset()

AssertionError: Torch not compiled with CUDA enabled

## 训练

In [ ]:
class Unet(DNN):
    """Physics Constrained Neural Networks
    """
    def __init__(self, dim_in=4, dim_out=1, dim_hidden=20, hidden_layers=5,
                 act_name='sigmoid', init_name=None):
        super().__init__(dim_in, dim_out, dim_hidden, hidden_layers,
                         act_name=act_name, init_name=init_name)
        self.model.fc5.bias.requires_grad = False
    def minmaxscaler(self, data):
        mmin = torch.min(data,dim=0)[0]
        mmax = torch.max(data,dim=0)[0]
        data = (data - mmin)/(mmax-mmin)
        output = data/torch.sum(data) * len(data)
        
        return output
    def forward(self, x):
        u = super().forward(x)
        u = self.minmaxscaler(u)
        return u

In [ ]:
class Trainer_AC(object):
    def __init__(self, args):
        self.model = args.model
        self.lr = args.lr
        self.gamma = args.gamma

        self.newton_iter = args.newton_iter
        self.step_size = args.step_size
        
        self.model_name = self.model.__class__.__name__
        self.model_path = self._model_path()
        
        self.epochs_Adam = args.epochs_Adam
        self.epochs_LBFGS = args.epochs_LBFGS
        self.optimizer_Adam = optim.Adam(self.model.parameters(), lr=self.lr, betas=(0.9, 0.999))
        self.optimizer_LBFGS = optim.LBFGS(self.model.parameters(),
                                            lr=0.8,
                                            max_iter=self.newton_iter,
                                            tolerance_grad=1.e-5,
                                            tolerance_change=1.e-9)
        self.scheduler = lr_scheduler.ExponentialLR(self.optimizer_Adam, gamma=self.gamma, verbose=True)
        
        # subnet(Unet)
        self.use_subnet = args.use_subnet
        self.Unet = args.Unet
        self.Unet.cuda()
        self.Unet.zero_grad()
        self.optimizer_Adam_Unet = optim.Adam(filter(lambda p: p.requires_grad==True, self.Unet.parameters()), lr=args.lr)    
        self.scheduler_Unet = lr_scheduler.ExponentialLR(self.optimizer_Adam_Unet, gamma=self.gamma, verbose=True)
        
        # data
        self.txy, self.txy_ics, self.u_ics = args.trainset()
 
        # Logger
        self.loss_log = []
        self.loss_b_log = []
        self.loss_r_log = []
        self.epoch_log = []
        
        self.loss_lbfgs_log = []
        self.loss_b_lbfgs_log = []
        self.loss_r_lbfgs_log = []
        self.epoch_lbfgs_log = []
        
        
    def _model_path(self):
        """Path to save the model"""
        if not os.path.exists('checkpoint'):
            os.mkdir('checkpoint')
        
        path = os.path.join('checkpoint', self.model_name)
        if not os.path.exists(path):
            os.mkdir(path)
        
        return path

    
    def net_r(self, txy):
        txy.requires_grad_(True)

        u = self.model(txy)

        grad_u = grad(u, txy)[0]
        u_t = grad_u[:,[0]]
        
        u_x = grad_u[:,[1]]
        u_y = grad_u[:,[2]]
        
        u_xx = grad(u_x, txy)[0][:,[1]]
        u_yy = grad(u_y, txy)[0][:,[2]]
        
        eps = 0.05
        lam = 10.
        
        residual = u_t - lam * (eps**2 * (u_xx + u_yy) - u**3 + u)

        return residual

    
    def net_u(self, txy):
        u = self.model(txy)
        
        return u
    
    
    def loss(self, use_ad=False):
        self.r_pred = self.net_r(self.txy)
        self.g_pred = self.net_u(self.txy_ics)

        if use_ad:

            self.r_weights = self.Unet(self.txy)
            loss_r = torch.mean(self.r_weights * self.r_pred**2)
            loss_b = torch.mean((self.g_pred-self.u_ics)**2)
            loss = loss_r + 100*loss_b

        else:
            loss_r = torch.mean(self.r_pred**2)
            loss_b = torch.mean((self.g_pred-self.u_ics)**2)
            loss = loss_r + 100*loss_b

        return loss, loss_r, loss_b


    def train_Adam(self, mode='0'):
        """
        mode='0'：subnet参与计算，只训练主网络
        mode='1'：subnet参与计算，只训练subnet
        mode='2'：subnet参与计算，主网络和subnet对抗训练
        mode='3'：subnet不参与计算，只训练主网络
        """
        if mode == '0':
            self.optimizer_Adam.zero_grad()

            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=True)
            loss_value.backward()
            self.optimizer_Adam.step()

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()
        
        elif mode == '1':
            self.optimizer_Adam_Unet.zero_grad()
            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=True)
            loss_value.backward()

            for name, parms in self.Unet.named_parameters():
                if parms.requires_grad==True:
                    parms.grad *= -1

            self.optimizer_Adam_Unet.step()        

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()

        elif mode == '2':
            self.optimizer_Adam.zero_grad()
            self.optimizer_Adam_Unet.zero_grad()

            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=True)
            loss_value.backward()
            self.optimizer_Adam.step()

            for name, parms in self.Unet.named_parameters():
                if parms.requires_grad==True:
                    parms.grad *= -1

            self.optimizer_Adam_Unet.step()

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()

        else:
            self.optimizer_Adam.zero_grad()

            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=False)
            loss_value.backward()
            self.optimizer_Adam.step()

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()
        

    def train_LBFGS(self):
        loss_value, loss_r_value, loss_b_value = self.loss()
        
        def closure():
            loss_value, loss_r_value, loss_b_value = self.loss()

            self.optimizer_LBFGS.zero_grad()
            loss_value.backward()

            return loss_value

        self.optimizer_LBFGS.step(closure)
        loss_value = closure()
        
        return loss_value.item(), loss_r_value.item(), loss_b_value.item()
    
    
    def validate(self, epoch):
        self.model.eval()
        loss_value, loss_r_value, loss_b_value = self.loss()
        
        infos = 'Valid   ' + \
                f'Loss:{loss_value:.4e}  ' + \
                f'Loss_r:{loss_r_value:.4e} '
        print(infos)
        self.model.train()
        return loss_value.item()
    
    
    def save_both_model(self, step, epoch):
        # save backbone
        model_state = {
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
        }
        backbone = os.path.join(self.model_path, 'backbone_%s_%d.pth' % (step, epoch))
        torch.save(model_state, backbone)

        # save subnet
        subnet_state = {
            'epoch': epoch,
            'state_dict': self.Unet.state_dict(),
        }    
        subnet = os.path.join(self.model_path, 'subnet_%s_%d.pth' % (step, epoch))
        torch.save(subnet_state, subnet)

    
    def train(self):
        start = time.time()
        best_loss = 1.e10
        
        # 1.只训练主网络
        for epoch in range(10000):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='3')    
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)
                
            if (epoch+1) % 100 == 0:          
                # data
                self.txy, self.txy_ics, self.u_ics = args.trainset()
                
        # 2.只训练subnet      
        for epoch in range(5000):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='1')      
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)
                
            if (epoch+1) % 100 == 0:      
                # data
                self.txy, self.txy_ics, self.u_ics = args.trainset()
                
                
        # 3.主网络和subnet对抗，一起训练      
        for epoch in range(self.epochs_Adam):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='2')
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            
            if (epoch+1) % self.step_size == 0:
                self.scheduler.step()
                self.scheduler_Unet.step()
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)  
                
            if (epoch+1) % 100 == 0:      
                # data
                self.txy, self.txy_ics, self.u_ics = args.trainset()
         
        
        # 使用最好的模型
        state_dict = torch.load(f'{self.model_path}/best_model.pth')
        self.model.load_state_dict(state_dict['state_dict'])
            
        # 4.只训练主网络主网络
        for epoch in range(self.epochs_Adam):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='3')
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            
            if (epoch+1) % self.step_size == 0:
                self.scheduler.step()         
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)  
                
            if (epoch+1) % 100 == 0:
                # data
                self.txy, self.txy_ics, self.u_ics = args.trainset()

        # 5.lbfgs只训练主网络主网络    
        for epoch in range(self.epochs_LBFGS):
            loss_value, loss_r_value, loss_b_value = self.train_LBFGS()
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)         
            
            if (epoch+1) % 1 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam + self.epochs_LBFGS}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)
                
            if (epoch+1) % 100 == 0:
                # data
                self.txy, self.txy_ics, self.u_ics = args.trainset()

In [ ]:
args.Unet = Unet(dim_in=4)
trainer = Trainer_AC(args)
trainer.train()

In [ ]:
# 使用最好的模型
state_dict = torch.load(f'{trainer.model_path}/best_model.pth')
model.load_state_dict(state_dict['state_dict'])

In [ ]:
file = f'checkpoint/subnet,{args.epochs_Adam}Adam,{args.epochs_LBFGS}LBFGS'
if os.path.exists(file)==False:
    os.mkdir(file)

In [ ]:
np.save(f'{file}/loss_r_log.npy',np.array(trainer.loss_r_log))
np.save(f'{file}/loss_b_log.npy',np.array(trainer.loss_b_log))

In [ ]:
trainer.loss_r_log = np.load(f'{file}/loss_r_log.npy')
trainer.loss_b_log = np.load(f'{file}/loss_b_log.npy')

In [ ]:
import seaborn as sns
with sns.axes_style("darkgrid"):
    fig = plt.figure(figsize=(6, 5))
    plt.rcParams.update({'font.size':16})
    plt.plot(np.arange(len(trainer.loss_r_log)), trainer.loss_r_log, label='$ \mathcal{L}_r$')
    plt.plot(np.arange(len(trainer.loss_b_log)), trainer.loss_b_log, label='$ \mathcal{L}_{ic}$')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.legend()

## 预测

In [ ]:
trainer.model.cpu().double();
trainer.Unet.cpu();

In [ ]:
x = np.linspace(0, 1, 51)
y = np.linspace(0, 1, 51)
t = np.linspace(0, 1, 11)
U_star = np.zeros((11, 51, 51))
# t = np.array([0,2.5,5,10])
# U_star = np.zeros((4, 51, 51))

In [ ]:
XX, YY = np.meshgrid(x, y)
XX = torch.from_numpy(XX).double()
YY = torch.from_numpy(YY).double()
nt = 11

U_pred_torch = torch.zeros([nt,XX.shape[0],XX.shape[1]]).double()

for i in range(nt):
    time = torch.ones([len(XX[0]), 1]) * t[i]
    for j in range(XX.shape[1]):
        txy = torch.hstack([time, YY[:,[j]], XX[:,[j]]])
        u_pred = trainer.net_u(txy)
        U_pred_torch[i][:,[j]] = u_pred.detach()

XX = XX.numpy()
YY = YY.numpy()
U_pred = U_pred_torch.numpy()

# error_u = np.linalg.norm(U_pred - U_star) / np.linalg.norm(U_star)
# print('u Relative l2 error: {:.3e}'.format(error_u))

In [ ]:
fig = plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
plt.pcolor(XX, YY, U_pred[0], vmin = -1, vmax =1)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$t=0$')
plt.colorbar()
plt.tight_layout()

plt.subplot(1, 3, 2)
plt.pcolor(XX, YY, U_pred[1], vmin = -1, vmax =1)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$t=1$')
plt.colorbar()
plt.tight_layout()

plt.subplot(1, 3, 3)
plt.pcolor(XX, YY, U_pred[2], vmin = -1, vmax =1)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$t=2$')
plt.colorbar()
plt.tight_layout()

plt.show()

In [ ]:
x = np.linspace(0, 1, 51)
y = np.linspace(0, 1, 51)
t = np.array([0,2.5,5,10])
U_star = np.zeros((4, 51, 51))

In [ ]:
XX, YY = np.meshgrid(x, y)
XX = torch.from_numpy(XX).double()
YY = torch.from_numpy(YY).double()
nt = 4

U_pred_torch = torch.zeros([nt,XX.shape[0],XX.shape[1]]).double()

for i in range(nt):
    time = torch.ones([len(XX[0]), 1]) * t[i]
    for j in range(XX.shape[1]):
        txy = torch.hstack([time, YY[:,[j]], XX[:,[j]]])
        u_pred = trainer.net_u(txy)
        U_pred_torch[i][:,[j]] = u_pred.detach()

XX = XX.numpy()
YY = YY.numpy()
U_pred = U_pred_torch.numpy()

# error_u = np.linalg.norm(U_pred - U_star) / np.linalg.norm(U_star)
# print('u Relative l2 error: {:.3e}'.format(error_u))

In [ ]:
fig = plt.figure(figsize=(24, 5))
plt.subplot(1, 4, 1)
plt.pcolor(XX, YY, U_pred[0], vmin = -1, vmax =1)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$t=0$')
plt.colorbar()
plt.tight_layout()

plt.subplot(1, 4, 2)
plt.pcolor(XX, YY, U_pred[1], vmin = -1, vmax =1)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$t=2.5$')
plt.colorbar()
plt.tight_layout()

plt.subplot(1, 4, 3)
plt.pcolor(XX, YY, U_pred[2], vmin = -1, vmax =1)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$t=5$')
plt.colorbar()
plt.tight_layout()

plt.subplot(1, 4, 4)
plt.pcolor(XX, YY, U_pred[3], vmin = -1, vmax =1)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$t=10$')
plt.colorbar()
plt.tight_layout()

plt.show()